# Irrigation Optimization

### Google Colab

In [ ]:
# !git clone https://github.com/EDSEL-skoltech/Intro_to_Digital_Agriculture.git
# %cd ./Intro_to_Digital_Agriculture/Optimization
# !pip install nevergrad
# !pip install pcse

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import json
import multiprocessing

from crop_model_en import Irrigation

class Optimization(Irrigation):
    import multiprocessing
    def __init__(self):
        super().__init__()
        self.year=None
        self.optimal_dates_irrigation = None
        self.num_process = 12
        
        self.irrigation_dates_for_many_years_optim = None
    def minimize_function_20_years_hpc(self, x):
        """
        Minimize this to define optimal day for irrigation
        """
        inputs_years = np.arange(self.NASA_start_year, self.NASA_start_year+20)
        #dates_irrigation = self.irrigation_dates(x)
        self.irrigation_dates_for_many_years_optim = self.irrigation_dates(x)
        
        pool = multiprocessing.Pool(processes=self.num_process)   
        crop_sim_for_20_years = pool.map(self.crop_hpc_20_years, inputs_years)
        yield_of_crop_sim_for_20_years = [crop_sim_for_20_years[i]['TWSO'][-1] for i in range(len(crop_sim_for_20_years))]

        out = np.mean(yield_of_crop_sim_for_20_years)
        return -out
    def minimize_function_20_years(self, x):
        """
        Minimize this to define optimal day for irrigation for 20 first years
        """
        inputs_years = np.arange(self.NASA_start_year, self.NASA_start_year+20)
        #dates_irrigation = self.irrigation_dates(x)
        self.irrigation_dates_for_many_years_optim = self.irrigation_dates(x)
        crop_sim_for_20_years = []
        for year in inputs_years:
            #change year from json-year to historical
            self.date_crop_start = self.year_changer(self.user_parameters['crop_start'],year)
            self.date_crop_end = self.year_changer(self.user_parameters['crop_end'],year)
            #convet dates from int to dt.datetime
            dates_irrigation = self.irrigation_dates(x)
            #Setup irrigation ammount
            amounts = [3. for _ in range(len(dates_irrigation))]
            dates_irrigation = [self.year_changer(obj, year) for obj in dates_irrigation]
            dates_npk, npk_list = self.user_parameters['npk_events'], self.user_parameters['npk']
            dates_npk = [self.year_changer(obj, year) for obj in dates_npk]
            agromanagement = self.agromanager_writer(self.user_parameters['crop_name'], dates_irrigation, dates_npk, amounts, npk_list)
            self.load_model()
            self.run_simulation_manager(agromanagement)
            output = pd.DataFrame(self.output).set_index("day")
            crop_sim_for_20_years.append(output)
        #select only last day crop yield 
        yield_of_crop_sim_for_20_years = [crop_sim_for_20_years[i]['TWSO'][-1] for i in range(len(crop_sim_for_20_years))]
        # calculate mean
        out = np.mean(yield_of_crop_sim_for_20_years)
        return -out    


## Nevergrad - A gradient-free optimization platform

<img src='./data/Nevergrad-LogoMark.png' alt="drawing" width=500/>

### Documentation : https://facebookresearch.github.io/nevergrad/

In [ ]:


import datetime as dt
import nevergrad as ng


import pandas as pd
import numpy as np
import json
import time
import multiprocessing
from concurrent import futures



number_of_iter = 1

budget = 20

optimizer = 'NGOpt4'


PATH_TO_USER_FILE='./util/input_data/malino_sugar_beet.json'

PATH_TO_NASA='./data/meteo/'
PATH_TO_DATA_DIR='./util/input_data/'
PATH_TO_RESULT_FILES = './optim/'

WOFOST = Optimization()


with open(PATH_TO_USER_FILE, 'r') as f:
    WOFOST.user_parameters = json.load(f)
WOFOST.num_process = 2
def round_geoposition(x, prec=1, base=.5):
    return round(base * round(float(x)/base),prec)
latitude = round_geoposition(WOFOST.user_parameters['latitude'])
longitude = round_geoposition(WOFOST.user_parameters['longitude'])
crop_name = WOFOST.user_parameters['crop_name']

WOFOST.weather_loader(PATH_TO_NASA, latitude, longitude)

WOFOST.data_dir = PATH_TO_DATA_DIR


year = 2017
WOFOST.year=year
dates_irrigation = WOFOST.user_parameters['irrigation_events']
crop_start = WOFOST.user_parameters['crop_start']
crop_start = WOFOST.year_changer(crop_start,year)
dates_irrigation = [WOFOST.year_changer(obj, year) for obj in dates_irrigation]
dates_irrigation = [dt.datetime.strptime(day, '%Y-%m-%d') for day in dates_irrigation]
dates_irrigation_integer = [(day-dt.datetime.strptime(crop_start,'%Y-%m-%d')).days for day in dates_irrigation]
print('Done!')
max_number_of_days = len(pd.date_range(start=WOFOST.user_parameters['crop_start'],end=WOFOST.user_parameters['crop_end']))
start = time.time()

names = [optimizer]
budget_for_optimizer = budget #200 or 500
path_to_result_files = PATH_TO_RESULT_FILES

for name in names:
    for i in range(number_of_iter):
        filepath = path_to_result_files+str(name)+'_number_'+str(i)+'_20_years'+'.txt'
        #Setup input model parameters
        instrum = ng.p.Tuple(*(ng.p.Scalar(lower=1, upper=max_number_of_days-10).set_integer_casting()for _ in range(len(dates_irrigation_integer))))
        
        #Select optimizer
        optimizer = ng.optimizers.registry[name](parametrization=instrum, budget=budget_for_optimizer, num_workers=1)
        
        #Logger to plots
        logger = ng.callbacks.ParametersLogger(filepath)
        optimizer.register_callback("tell",  logger)
        
        #Show progress
        optimizer.register_callback("tell", ng.callbacks.ProgressBar())

        
        #Start minimization
        recommendation = optimizer.minimize(WOFOST.minimize_function_20_years)
        with open(path_to_result_files+str(name)+'_number_'+str(i)+".json", "w") as fp:    
            json.dump(logger.load(), fp)

        resulted_optim_info = str(recommendation.value)+', '+'Time:'+str(time.time()-start)+', '+'budget:'+str(optimizer.budget)+'\n'
        with open(path_to_result_files+'Result'+str(name)+'.txt', 'a') as msk:
            msk.write(resulted_optim_info)
print('Optimizers: Finish! Time for exec:', time.time()-start)